# Jeopardy Topics

In [ ]:
import numpy as np
import spacy
nlp = spacy.load('en_core_web_sm')
import pandas as pd
pd.set_option('display.max_colwidth', 800)

In [ ]:
JEOPARDY_CSV =  '../data/jeopardy/Jeopardy.csv'

questions = pd.read_csv(JEOPARDY_CSV)

questions.columns = [x.strip() for x in questions.columns]

In [ ]:
questions = questions.dropna(subset=['Question'])

In [ ]:
questions['Category'].nunique()

In [ ]:
file='../data/jeopardy/JQuestions.txt'
questions['Question'].sample(frac=0.04,replace=False,random_state=0).to_csv(file)

f=open(file,'r',encoding='utf-8')
text=f.read()
f.close()

doc=nlp(text)
pos_list=['NOUN']
preproc_text=[]
preproc_sent=[]

for token in doc:
    if token.text!='\n':
        if not(token.is_stop) and not(token.is_punct) and token.pos_ in pos_list:
            preproc_sent.append(token.lemma_)
    else:
        preproc_text.append(preproc_sent)
        preproc_sent=[]

preproc_text.append(preproc_sent) #last sentence

print(preproc_text)

In [ ]:
import tomotopy as tp
NUM_TOPICS=1000

mdl = tp.LDAModel(k=NUM_TOPICS,seed=1234)

for line in preproc_text:
    mdl.add_doc(line)
    
mdl.train(10)
    
for k in range(mdl.k):
    print('Top 7 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=7))

In [ ]:
print('Log perplexity=',mdl.ll_per_word)

In [ ]:
bag_of_words=[word for sent in preproc_text for word in sent]
doc_inst = mdl.make_doc(bag_of_words)
np.argsort(np.array(mdl.infer(doc_inst)[0]))[::-1]

In [ ]:
print(mdl.get_topic_words(461, top_n=7))

In [ ]:
print(mdl.get_topic_words(234, top_n=7))

In [ ]:
print(mdl.get_topic_words(186, top_n=7))